# Example: Add priors to your objectiv function¶
This is an example using the simple model [priorModel.xml] to demonstrate how you can add priors to your objective function

# Model import

In [60]:
import libsbml
import importlib
import amici
import pypesto
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

In [61]:
#import SBML model
sbml_file = 'priorModel.xml'
model_name = 'priorModel'
model_output_dir = model_name

sbml_reader = libsbml.SBMLReader()
sbml_doc = sbml_reader.readSBML(sbml_file)
sbml_model = sbml_doc.getModel()
dir(sbml_doc)

sbml_importer = amici.SbmlImporter(sbml_file)
sbml = sbml_importer.sbml

In [62]:
# Retrieve model output names and formulae from AssignmentRules and remove the respective rules
observables = amici.assignmentRules2observables(
         sbml_importer.sbml,
         filter_function = lambda variable: variable.getId().startswith('observable_')
         )

sbml_importer.sbml2amici(model_name,
                         model_output_dir,
                         verbose=False,
                         observables=observables)

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)

model = model_module.getModel()

In [63]:
# Show the model
print("Model parameters:", list(model.getParameterIds()))
print("Model outputs:   ", list(model.getObservableIds()))
print("Model states:    ", list(model.getStateIds()))

print('\nReactions:')
for reaction in sbml_model.getListOfReactions():
    reactants = ' + '.join(['%s %s'%(int(r.getStoichiometry()) if r.getStoichiometry() > 1 else '', r.getSpecies()) for r in reaction.getListOfReactants()])
    products  = ' + '.join(['%s %s'%(int(r.getStoichiometry()) if r.getStoichiometry() > 1 else '', r.getSpecies()) for r in reaction.getListOfProducts()])
    reversible = '<' if reaction.getReversible() else ''
    print('%3s: %10s %1s->%10s\t\t[%s]' % (reaction.getId(),
                        reactants,
                        reversible,
                         products,
                        libsbml.formulaToL3String(reaction.getKineticLaw().getMath())))

Model parameters: ['k1', 'k2', 'k3', 'noiseparameter1_x2']
Model outputs:    ['observable_x2']
Model states:     ['x1', 'x2']

Reactions:
 r1:             ->        x1		[c1 * function_for__r1__stateModel(c1, k1)]
 r2:         x1  ->        x2		[c1 * function_for_r2__stateModel(x1, c1, k2)]
 r3:   x1 +  x2  ->          		[c1 * function_for_r3__stateModel(k3, x1, x2, c1)]


# Define Prior

To define your prior, you need 2(4) Inputs:
    
    1) priorType_list: list wity types of the prior: 'lap' = laplacian prior, 'norm' = normal prior
    
    2) priorParameters_list: list with the Parameters of the different priors:
    laplacian prior: [[mean,scale],[mean,scale],...]
    normal prior: [[mean,var],[mean,var],...]
    
    OPTIONAL:
    
        3) estimate_list: array of 0 and 1. 1 means you want to estimate (add prior information) to this parameter. DEFAULT: all parameters get estimated
    
    4) scale_list: list with scaling types of the parameters: 'lin' (default), 'log', 'logE' (=log10(1+x))
    
In the future with the definition of the new input format, all this inputs should get build automatically from the files.

In [64]:
# define Prior parameters
priorType_list = ['lap','lap','lap','lap']
priorParameter_list = [[0,1/10],[0,1/10],[0,1/10],[0,1/10]]

# using 2 different parameter scales
scale_list_lin = ['lin','lin','lin','lin']
scale_list_log = ['log10','log10','log10','log10']

# we want to estimate the parameters k_1,k_2,k_3 and not the noiseparameter
estimate_list = np.array([1,1,1,0])

In [65]:
# define the prior
prior_lin = pypesto.Prior(priorType_list=priorType_list,
                             priorParameters_list=priorParameter_list,
                             estimate_list=estimate_list,
                             scale_list=scale_list_lin)

prior_log = pypesto.Prior(priorType_list=priorType_list,
                             priorParameters_list=priorParameter_list,
                             estimate_list=estimate_list,
                             scale_list=scale_list_log)

# evaluate the prior at x
x_lin = np.array([1,2,3,4])
x_log = np.log10(x_lin)

print(prior_lin(x=x_lin,sensi_orders=1))
print(prior_log(x=x_log,sensi_orders=1))

{'prior_fun': -55.171686262697705, 'prior_grad': array([-10., -10., -10.,   0.]), 'chainrule': array([1., 1., 1., 0.])}
{'prior_fun': -55.171686262697705, 'prior_grad': array([-23.02585093, -46.05170186, -69.07755279,   0.        ]), 'chainrule': array([2.30258509, 4.60517019, 6.90775528, 0.        ])}


# Add prior to your objective Function

1) Generate objective function

In [66]:
# set timepoints for which we want to simulate the model
model.setTimepoints(amici.DoubleVector(np.linspace(0, 60, 10)))

# set parameter Scale
#model.setParameterScale(amici.ParameterScaling_log10)
model.setParameterScale(amici.ParameterScaling_none)

#simulate the true model (k_3=0)
model.setParameters(amici.DoubleVector([0.2,0.4,0,0.2]))

# Create solver instance
solver = model.getSolver()

# enable sensitivities
solver.setSensitivityOrder(amici.SensitivityOrder_first)       # First-order ...
solver.setSensitivityMethod(amici.SensitivityMethod_forward)   # ... forward sensitivities
model.requireSensitivitiesForAllParameters()                   # ... w.r.t. all parameters

# Run simulation using default model parameters and solver options and without experimental data
rdata = amici.runAmiciSimulation(model, solver)

# Create ExpData instance from simulation results (noise = 0.2)
edata = amici.ExpData(rdata, 0.2, 0.0)

# Re-run simulation, this time passing "experimental data"
rdata = amici.runAmiciSimulation(model, solver, edata)

In [67]:
#generate objective function from amici model
objective = pypesto.AmiciObjective(model, solver, [edata], 1)

2) Add priors

In [68]:
# Linear case

# objective with linear prior
objective_lin = pypesto.Objective(fun=objective.get_fval, grad=objective.get_grad, prior=prior_lin)

In [69]:
# Log case, we have to manipulate the inputs

# objective with prior logE
obj_log  = lambda x: objective.get_fval(10**x)
grad_log = lambda x: objective.get_grad(10**x)

objective_logE = pypesto.Objective(fun=obj_log, grad=grad_log, prior=prior_log)

In [70]:
# OUTPUTS

# evaluate objective function at y
y_lin = amici.DoubleVector([1,2,3,4])
y_log = amici.DoubleVector([np.log10(1),np.log10(2),np.log10(3),np.log10(4)])

print('\nobjectiv funciton without prior:', objective.get_fval(y_lin))
print('objectiv function with l1 (lin):', objective_lin.get_fval(y_lin))
print('objectiv function with l1 (log):', objective_logE.get_fval(y_log))


objectiv funciton without prior: 7378.431290046302
objectiv function with l1 (lin): 7433.602976309
objectiv function with l1 (log): 7433.602976309
